In [13]:
from __future__ import division

In [14]:
import csv
import pandas as pd
import datetime
import string
import numpy as np
import helper as hp
import sq
import jp

In [15]:
Gb = 1000000000

In [16]:
#store = pd.HDFStore('access_log.h5')

In [17]:
def get_server_from_URL(URLs):
    result = []
    
    for u in URLs:
        if u.startswith('http://'):
            temp = u.split('http://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
        elif u.startswith('http://'):
            temp = u.split('https://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
    return result

In [18]:
def remove_safe_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True
                break
                
        if (m == False):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result            

In [19]:
def get_match_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True                
                break
                                
        if (m == True):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result   

In [20]:
#result = []

#f = hp.open_tar_gz('sgkc-ssg140.log-20150610.tar.gz')

#f = f[0].split('\n')

In [21]:
#sample = f[0]
#sample

In [22]:
def process_juniper_line(line):
    result = []
    line = hp.remove_duplicate_spaces(line).split(' ')
    result.append(line[0] + ' ' + line[1] + ' ' + line[2])
    result.append(line[3])
    result.append(line[4] + ' ' + line[5])
    result.append(line[6])
    result.append(line[8] + ' ' + line[9])
    for i in range(10, 13):
        result.append(line[i])
    result.append(line[14] + ' ' + line[15])    
    result.append(line[16] + ' ' + line[17])
    for i in range(18, 25):
        result.append(line[i])
    result.append(line[26] + ' ' + line[27]) 
    result.append(line[28] + ' ' + line[29]) 
    for i in range(30, len(line) - 4):
        result.append(line[i])
    result.append(line[32] + ' ' + line[33] + ' ' + line[34] + ' ' + line[35])
    return result

In [23]:
squid = sq.read_squid_log('access.log')

In [24]:
squid.tail()

,date,duration,client_address,result_code,bytes,method,URL,hierarchy,type
521551,2015-09-17 03:45:58.797000,0,172.20.78.98,TCP_DENIED/403,3794,GET,http://js.tongji.linezing.com/2112694/tongji.js,HIER_NONE/-,text/html
521552,2015-09-17 03:45:59.384000,1,172.20.78.167,TCP_DENIED/403,4026,GET,http://csm90-en.url.trendmicro.com/T/152/Q0c96...,HIER_NONE/-,text/html
521553,2015-09-17 03:46:00.733000,4736,172.20.114.37,TCP_MISS/200,0,CONNECT,www.bdpinsight.eu:443,HIER_DIRECT/190.93.241.58,-
521554,2015-09-17 03:46:00.734000,0,172.20.78.167,TCP_DENIED/403,3892,GET,http://csm90-en.url.trendmicro.com/T/108/2uHnX...,HIER_NONE/-,text/html
521555,2015-09-17 03:46:01.632000,0,172.20.78.167,TCP_DENIED/403,4142,GET,http://csm90-en.url.trendmicro.com/T/192/Q0c96...,HIER_NONE/-,text/html


In [25]:
safe_websites = 'google|microsoft|trendmicro|gstatic.com|bdpinsight.eu|.gov.hk'

In [26]:
access = squid[squid['URL'].str.contains(safe_websites) == False]

In [27]:
sum(squid.bytes)/Gb

15.718760193

In [28]:
#store.put('GZ', squid, format='table')
#store

In [29]:
#df = []

In [30]:
#store.open()

In [31]:
#pd.read_table

In [32]:
#squid.h

In [33]:
#np.max(squid['bytes'])

In [34]:
#squid.head()

In [35]:
#URL = squid[squid['time'] > '2015-06-05 09:00:00']['URL']
#URL = squid[squid['client_address'] == '172.20.70.89']['URL']
#URL = squid['URL']

In [36]:
#server = get_server_from_URL(URL)

In [37]:
#question_url = remove_safe_site(server, ['qq.com', 'bdpint.com', 'sogou.com', 'sohu.com', 'trendmicro.com'])

In [38]:
#np.mean(squid['bytes'])

In [39]:
#squid.to_csv('20150601004228-20150608105645.csv', index=False)

In [70]:
def read_syslog_juniper(filename, filter=None):
    result = []
    
    #if (filename)[-3:] == '.gz':
    #    f = hp.open_gz(filename)
    #elif (filename)[-6:] == 'tar.gz':
    #    f = hp.open_tar_gz(filename)
    #else:
    #    f = hp.open_txt(filename)
    
    #f = f.split('\n')
    
    df = pd.read_table(filename, sep=' {1,}', engine='python', index_col=False, header=None, parse_dates=[0])
    
    #return df
    return df

In [71]:
jp = read_syslog_juniper('hk-ssg140.1.log')

In [72]:
jp

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:52""",...,ip=172.20.68.77,port=50863,dst-xlated,ip=172.20.113.3,port=443,session_id=40329,reason=Close,-,TCP,RST
1,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:54""",...,ip=172.20.78.2,port=2490,dst-xlated,ip=172.20.78.20,port=8059,session_id=38500,reason=Close,-,TCP,FIN
2,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:53""",...,code=0,src-xlated,ip=172.24.66.31,dst-xlated,ip=172.20.78.35,session_id=37797,reason=Close,-,RESP,None
3,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:55""",...,ip=202.77.48.25,port=10926,dst-xlated,ip=202.14.67.4,port=53,session_id=42617,reason=Close,-,RESP,None
4,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:55""",...,ip=202.77.48.25,port=21265,dst-xlated,ip=192.48.79.30,port=53,session_id=37258,reason=Close,-,RESP,None
5,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:55""",...,ip=202.77.48.25,port=16469,dst-xlated,ip=8.8.8.8,port=53,session_id=45254,reason=Close,-,RESP,None
6,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:55""",...,ip=202.77.48.25,port=31815,dst-xlated,ip=208.109.255.8,port=53,session_id=44427,reason=Close,-,RESP,None
7,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:55""",...,ip=202.77.48.25,port=13809,dst-xlated,ip=202.153.97.2,port=53,session_id=40056,reason=Close,-,RESP,None
8,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:55""",...,ip=202.77.48.25,port=24371,dst-xlated,ip=216.69.185.8,port=53,session_id=47719,reason=Close,-,RESP,None
9,2015-09-18,18,03:34:04,172.20.68.4,hk-SSG140:,NetScreen,device_id=hk-SSG140,[Root]system-notification-00257(traffic):,"start_time=""2015-09-18","03:45:55""",...,ip=202.77.48.25,port=10948,dst-xlated,ip=210.184.16.65,port=53,session_id=46720,reason=Close,-,RESP,None
